# Part 2. Генерация текстов
## Белова Полина, Фролова Юлия
### Тексты М.А.Булгаков: тренировочный - "Собачье сердце", тестовый - "Роковые яйца"

In [1]:
file1 = open('bulg2.txt', 'r', encoding='utf8')
bulg1 = file1.readlines()
file1.close()
file2= open('bulg1.txt', 'r', encoding='utf8')
bulg2 = file2.readlines()
file2.close()

In [2]:
from nltk.tokenize import word_tokenize

In [3]:
bulg1 = ' '.join(bulg1)
bulg2 = ' '.join(bulg2)

убираем все знаки препинания, числа и прочие непонятные знаки. Не хватало памяти на знаки препинания

In [4]:
words1 = [w.lower() for w in word_tokenize(bulg1) if w.isalpha()]
words2 = [w.lower() for w in word_tokenize(bulg2) if w.isalpha()]

убираем незнакомые слова в тестовом тексте

In [6]:
words_new = []
for i in words2:
    if i in words1:
        words_new.append(i)

In [7]:
words2 = words_new

In [8]:
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Activation
import numpy as np
from sklearn.model_selection import train_test_split

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
words = set([w for w in words1]) # убираем повторяющиеся слова
nb_words = len(words)
word2index = dict((w, i) for i, w in enumerate(words)) # присваиваем каждому слову уникальное число
index2word = dict((i, w) for i, w in enumerate(words)) # то же самое, но наоборот

In [10]:
SEQLEN = 10 # длина отрезка, который мы подаем 
STEP = 1 # будем добавлять по одному слову

input_words = [] # список всех возможных последовательностей из 10 слов
label_words = [] # список их продолжений
test_word = []
test_label = []
for i in range(0, len(words1) - SEQLEN, STEP):
    input_words.append(words1[i:i + SEQLEN])
    label_words.append(words1[i + SEQLEN])
for i in range(0, len(words2) - SEQLEN, STEP):
    test_word.append(words2[i:i + SEQLEN])
    test_label.append(words2[i + SEQLEN])

разбиваем списки из тренировочного текста на тренировочную и валидационную выборки

In [11]:
X = np.zeros((len(input_words), SEQLEN))
y = np.zeros((len(input_words), nb_words))
for i,ch in enumerate(input_words):
    for j, cm in enumerate(ch):
        X[i, j] = word2index[cm]
    y[i, word2index[label_words[i]]] = 1
Xtrain, Xval, Ytrain, Yval = train_test_split(X, y, test_size = 0.2)

In [14]:
import math

высчитываем перплексию

In [15]:
def perplexity(probabilities):
    sumlog = 0
    for probability in probabilities:
        log_prob = math.log2(probability)
        sumlog += log_prob
    l = sumlog/len(probabilities)
    perp = math.pow(2, -l)
    return perp

In [16]:
def evaluate_model(model):
    for i in range(len(test_word)): 
        probabilities = []
        test_input = test_word[i]
        test_l = test_label[i]
        test_s = np.zeros((1, SEQLEN))
        for w, word in enumerate(test_input):
            test_s[0, w] = word2index[word] # кодируем кажду последовательность в тестовой выборке, чтобы по ней потом предсказать
        pred = model.predict(test_s, verbose=0)[0]
        probability = pred[word2index[test_l]] # ищем вероятность с нужного нам индекса
        probabilities.append(probability)
    perp = perplexity(probabilities)
    return perp

In [18]:
import random

пишем текстик на основе какой-нибудь последовательности из тестовой выборки

In [19]:
def text(model):
    start = np.random.randint(len(test_word)) #рандомно выбираем индекс тестовой последовательности
    test_input = test_word[start] # берем эту последовательность для начала текста
    print(' '.join(test_input))
    for step in range(200): # добавим к рандомно выбранным 10-ти словам еще 200
        test_s = np.zeros((1, SEQLEN))
        for w, word in enumerate(test_input):
            test_s[0, w] = word2index[word]
        pred = model.predict(test_s, verbose=0)[0]
        vs = [] # варианты будущего слова
        for i in range(5): # из пяти слов с максимально вероятностью следования после нашей последовательности
            max_idx = np.argmax(pred) # находим индекс максимально вероятности
            pred_w = index2word[max_idx]  # находим это слово
            vs.append(pred_w) # добавляем в варианты
            pred[max_idx] = 0
        new_w = random.choice(vs) # рандомно выбираем один из пяти
        print(new_w, end=' ')
        test_input = test_input[1:] + [new_w]
    print()

обучаем модельки и смотрим, какие результаты они покажут

### LSTM Emdedding 50

In [12]:
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 10
NUM_PREDS_PER_EPOCH = 100

lstm_model = Sequential()
lstm_model.add(Embedding(nb_words, 50, input_length=SEQLEN))
lstm_model.add(LSTM(HIDDEN_SIZE, return_sequences=False,
                    input_shape=(SEQLEN, nb_words),
                    unroll=True))
lstm_model.add(Dense(nb_words))
lstm_model.add(Activation("softmax"))

lstm_model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=['acc'])

In [13]:
lstm_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 50)            398700    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dense_1 (Dense)              (None, 7974)              1028646   
_________________________________________________________________
activation_1 (Activation)    (None, 7974)              0         
Total params: 1,518,994
Trainable params: 1,518,994
Non-trainable params: 0
_________________________________________________________________


In [14]:
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    lstm_model.fit(Xtrain, Ytrain, validation_data=[Xval, Yval], batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)
    
    test_idx = np.random.randint(len(input_words))
    test_input = input_words[test_idx]
    print(' '.join(test_input))
    for i in range(NUM_PREDS_PER_EPOCH):
        test_s = np.zeros((1, SEQLEN))
        for i, w in enumerate(test_input):
            test_s[0, i] = word2index[w]
        pred = lstm_model.predict(test_s, verbose=0)[0]
        ypred = index2word[np.argmax(pred)]
        print(ypred, end=" ")
        test_input = test_input[1:] + [ypred]
    print()

Iteration #: 0
Train on 18998 samples, validate on 4750 samples
Epoch 1/10
18998/18998 [==============================] - 25s 1ms/step - loss: 7.9527 - acc: 0.0378 - val_loss: 7.9086 - val_acc: 0.0406
Epoch 2/10
18998/18998 [==============================] - 25s 1ms/step - loss: 7.5666 - acc: 0.0383 - val_loss: 8.0408 - val_acc: 0.0406
Epoch 3/10
18998/18998 [==============================] - 20s 1ms/step - loss: 7.4334 - acc: 0.0405 - val_loss: 8.0355 - val_acc: 0.0524
Epoch 4/10
18998/18998 [==============================] - 21s 1ms/step - loss: 7.3270 - acc: 0.0477 - val_loss: 8.1050 - val_acc: 0.0524
Epoch 5/10
18998/18998 [==============================] - 31s 2ms/step - loss: 7.2359 - acc: 0.0497 - val_loss: 8.0479 - val_acc: 0.0531
Epoch 6/10
18998/18998 [==============================] - 30s 2ms/step - loss: 7.1544 - acc: 0.0505 - val_loss: 8.0100 - val_acc: 0.0518
Epoch 7/10
18998/18998 [==============================] - 30s 2ms/step - loss: 7.0784 - acc: 0.0532 - val_loss: 8.

сам сам буду моя в ноги но здесь шариков я почему почему но он не бок не ответил ли шариков шариков раз ничего и больше голос раз трубку в петровны вами пса раз дверь из вами очередь да борменталь том он воскликнул он так не головой и же вы вас уж чёрный и я я быть о будут иван что бы или вас не было доктор их я спросил борменталь вам ни я вам лет я иван очень они туда человек ним их только бы ничего всё будет за совершенно ним этого как к себе филипповичем здесь того шариков где 
Iteration #: 8
Train on 18998 samples, validate on 4750 samples
Epoch 1/10
18998/18998 [==============================] - 32s 2ms/step - loss: 3.4754 - acc: 0.5474 - val_loss: 10.0610 - val_acc: 0.0280
Epoch 2/10
18998/18998 [==============================] - 31s 2ms/step - loss: 3.4710 - acc: 0.5484 - val_loss: 10.0248 - val_acc: 0.0288
Epoch 3/10
18998/18998 [==============================] - 32s 2ms/step - loss: 3.4673 - acc: 0.5511 - val_loss: 10.0080 - val_acc: 0.0267
Epoch 4/10
18998/18998 [============

передёрнуло чёрт документ а может быть это голос его звучал
был и тебе преображенский уверенно ответил его он вас того профессор себя шариков так ну и был кухне филипп филиппович одного болезни полиграф где ванной под вот может что при в меня квартире из смотровой ко вот и севильи вас он за в передней вы профессор вы женщины или не пока не профессор шариков вам нельзя ну они филипп филиппович вам с вас а быть нет вы вы не раз уж шариков мой не на глазами могу раз туда знает лицо тебе руках вы говорю на совершенно где вы его женщины во и себя вы глаз того филипп филиппович мне 
Iteration #: 12
Train on 18998 samples, validate on 4750 samples
Epoch 1/10
18998/18998 [==============================] - 32s 2ms/step - loss: 3.4426 - acc: 0.5905 - val_loss: 10.1072 - val_acc: 0.0240
Epoch 2/10
18998/18998 [==============================] - 32s 2ms/step - loss: 3.4397 - acc: 0.5909 - val_loss: 10.0889 - val_acc: 0.0263
Epoch 3/10
18998/18998 [==============================] - 32s 2ms/step - lo

Epoch 9/10
18998/18998 [==============================] - 26s 1ms/step - loss: 3.2925 - acc: 0.6197 - val_loss: 10.5688 - val_acc: 0.0213
Epoch 10/10
18998/18998 [==============================] - 25s 1ms/step - loss: 3.2914 - acc: 0.6193 - val_loss: 10.5777 - val_acc: 0.0219
отпер парадную дверь и впустил шарика зине он при этом
заметил тяпнутый оба ошейник голос мне есть полагается рукой а а вот фёдор о но этого в них угодно что шариков ну это человек они более потом двери слова к была пальто разве на на штаны болезни дня бок от под ночь до а разве на была появилась бок и глаз в бросился ночь доктор тотчас тотчас оба всей хоть в нос шариков пока позвольте от всего подумал мне не вынул я лестнице после лицо чего только да вы у смотровой тем имени не преображенский фёдор вы будут одна одна лицо значит никакой и подал во бок по моей со 
Iteration #: 20
Train on 18998 samples, validate on 4750 samples
Epoch 1/10
18998/18998 [==============================] - 27s 1ms/step - loss: 3.2896 -

In [17]:
evaluation = evaluate_model(lstm_model)
print(evaluation)

80.23315832470844


In [20]:
something = text(lstm_model)

у него и жарко по и рукой профессор за перила
с субъект пусть несмотря филипп филиппович даже в головой к подал над часа их говорил хотел трубку начал в которого лет спросил филипп они же вас сейчас меня есть вас так филипп и закричал филиппович же не это не понимаю слово он арнольдович нежно лицо о моей господа господа себя глазами очень поднялся глаза голову пса он этот этот опять лет по сего сидел человека профессор до оперировать при ванной нос вслед роста его говорит зеркало а у раз в говорите вдруг я после шарикову до если вдруг от моей сквозь ему не только за я говорит пока я не вы столе разве я поднял это её его можете вместо людей с общем пока филипп а ему она мне быть между он шариков он поднял о хорошо вечером борменталь ничего женщины дверью доктор это нужно клим ему ночь то туда филипп с борменталь нужно может рот тот вышел дверь шарикова глазами знаете от довольно можете себя чтобы быть будто хорошо теперь за сердце о ему думал не довольно женщины вам передней я позвольте

### LSTM Embedding 100

In [30]:
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 10
NUM_PREDS_PER_EPOCH = 100

lstm_model = Sequential()
lstm_model.add(Embedding(nb_words, 100, input_length=SEQLEN))
lstm_model.add(LSTM(HIDDEN_SIZE, return_sequences=False,
                    input_shape=(SEQLEN, nb_words),
                    unroll=True))
lstm_model.add(Dense(nb_words))
lstm_model.add(Activation("softmax"))

lstm_model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=['acc'])

In [31]:
lstm_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 10, 100)           797400    
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_4 (Dense)              (None, 7974)              1028646   
_________________________________________________________________
activation_4 (Activation)    (None, 7974)              0         
Total params: 1,943,294
Trainable params: 1,943,294
Non-trainable params: 0
_________________________________________________________________


In [32]:
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    lstm_model.fit(Xtrain, Ytrain, validation_data=[Xval, Yval], batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)
    
    test_idx = np.random.randint(len(input_words))
    test_input = input_words[test_idx]
    print(' '.join(test_input))
    for i in range(NUM_PREDS_PER_EPOCH):
        test_s = np.zeros((1, SEQLEN))
        for i, w in enumerate(test_input):
            test_s[0, i] = word2index[w]
        pred = lstm_model.predict(test_s, verbose=0)[0]
        ypred = index2word[np.argmax(pred)]
        print(ypred, end=" ")
        test_input = test_input[1:] + [ypred]
    print()

Iteration #: 0
Train on 18998 samples, validate on 4750 samples
Epoch 1/10
18998/18998 [==============================] - 46s 2ms/step - loss: 7.9453 - acc: 0.0362 - val_loss: 7.8594 - val_acc: 0.0411
Epoch 2/10
18998/18998 [==============================] - 30s 2ms/step - loss: 7.5401 - acc: 0.0387 - val_loss: 7.9494 - val_acc: 0.0404
Epoch 3/10
18998/18998 [==============================] - 29s 2ms/step - loss: 7.3899 - acc: 0.0480 - val_loss: 7.9902 - val_acc: 0.0535
Epoch 4/10
18998/18998 [==============================] - 30s 2ms/step - loss: 7.2726 - acc: 0.0486 - val_loss: 8.1004 - val_acc: 0.0491
Epoch 5/10
18998/18998 [==============================] - 30s 2ms/step - loss: 7.1756 - acc: 0.0494 - val_loss: 8.0414 - val_acc: 0.0537
Epoch 6/10
18998/18998 [==============================] - 30s 2ms/step - loss: 7.0793 - acc: 0.0550 - val_loss: 7.9742 - val_acc: 0.0533
Epoch 7/10
18998/18998 [==============================] - 30s 2ms/step - loss: 6.9819 - acc: 0.0569 - val_loss: 8.

18998/18998 [==============================] - 29s 2ms/step - loss: 3.4353 - acc: 0.5832 - val_loss: 9.8620 - val_acc: 0.0261
движение ничего я не понимаю ответил филипп филиппович королевски вздёргивая
не какого за шарикова очень которого очень воскликнул шариков я он где его молча в приёмной мозгу филиппа филиппович шариков с и спросил филипп филиппович вы в квартире просто просто что вы вы её да что будет квартире по ли быть ещё человек вас не не но но на вы мы не только не ко на не позвольте и все ним же и тут из ним ну за квартиры а теперь хорошо я из него что тут человек нельзя же доктор я голубчик впрочем вы вы профессор не её филипп филиппович но это время раз голос за шариков квартиры я 
Iteration #: 8
Train on 18998 samples, validate on 4750 samples
Epoch 1/10
18998/18998 [==============================] - 28s 1ms/step - loss: 3.4314 - acc: 0.5843 - val_loss: 9.8422 - val_acc: 0.0240
Epoch 2/10
18998/18998 [==============================] - 28s 1ms/step - loss: 3.4230 - acc: 

Epoch 10/10
18998/18998 [==============================] - 29s 2ms/step - loss: 3.4738 - acc: 0.6056 - val_loss: 10.0977 - val_acc: 0.0211
не сверкнул над ним красным лучом очень настойчиво с залихватской
день жизнь того бросился из квартире но их вас приёмную что по мне москве мне ведь что вас вас что может потолком я один они у когда вдруг стене люди пёс на меня не сколько её когда ли он будет пять пять без человек голосом просто к ещё а раз могу опять за нельзя же более стене глухо очень вечером вас ну меня когда ещё к совершенно шарикову сейчас закричал на без же сам головой отозвался на известно это я не буду говорю о за вы нельзя я впрочем первый и очень раз на шарикова но сегодня при фёдор в 
Iteration #: 12
Train on 18998 samples, validate on 4750 samples
Epoch 1/10
18998/18998 [==============================] - 28s 1ms/step - loss: 3.4755 - acc: 0.6043 - val_loss: 10.0895 - val_acc: 0.0227
Epoch 2/10
18998/18998 [==============================] - 29s 2ms/step - loss: 3.4740 - 

Epoch 8/10
18998/18998 [==============================] - 28s 1ms/step - loss: 3.4342 - acc: 0.6224 - val_loss: 10.5403 - val_acc: 0.0225
Epoch 9/10
18998/18998 [==============================] - 28s 1ms/step - loss: 3.4342 - acc: 0.6224 - val_loss: 10.5261 - val_acc: 0.0219
Epoch 10/10
18998/18998 [==============================] - 28s 1ms/step - loss: 3.4366 - acc: 0.6230 - val_loss: 10.5655 - val_acc: 0.0219
преображенской заставы осколки солнца в бутылках битый кирпич вольные псы
человек вот куда уж ни на доме ответил водки не спрашивается человек мозг в в этот потом но но донёсся пёс а его с мы хорошо ноги хорошо ну фёдор ничего сердце я а зачем был вот вот когда знает что впрочем за лет же будем и начал сударыня начал филипп филиппович же у у одним него воскликнул вас борменталь не какая и ноги и угодно ответил ответил филиппович вам вас её извините уж не глазами вам можете самого закричал делать профессор с это не угодно существо профессора потому вам сердце ко её ты по и профес

In [42]:
evaluation = evaluate_model(lstm_model)

In [43]:
print(evaluation)

25739.492306372722


In [44]:
something = text(lstm_model)

нам под подпись свет над на выскочили слова газета и
в узнать его пса нужно своим заметил своим ест червонца господа ах вот именно у он мгновенно отсюда впрочем или ним и ещё лапы спрашивается пригласил череп чего могу будет дверь борменталя я по лестнице послышался стоял оказался да голос такой не глянул нашего мозг какой он просто отозвался он же я на кресле в дама ей которого что а осведомился что был и прошу тебя молча какой на этой квартиры мой как сегодня ни позвольте именно без не по сказать окончательно оказался что это ест двери молвил а случае непосредственно лет взял из какой доктор живот мне пёс кто личность ни нельзя почему все не не лет напрасно где свою на о звонок этом когда начал в преображенский слова эта вошёл ну квартиры или к то произнёс сам по часа калоши в севильи про пёс насчёт да человек прошу осведомился если что если дарья пальто осведомился видели тот так кого бы будто вы я как гражданин спросил три господин по затем сказать сам профессор говорить дарья фили

### LSTM Embedding 50 Hidden_size = 216 

In [34]:
HIDDEN_SIZE = 216 #опечатка вышла при заполнении кол-ва нейронов, предполагалось 256, но уже посчиталось
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 10
NUM_PREDS_PER_EPOCH = 100

lstm_model = Sequential()
lstm_model.add(Embedding(nb_words, 50, input_length=SEQLEN))
lstm_model.add(LSTM(HIDDEN_SIZE, return_sequences=False,
                    input_shape=(SEQLEN, nb_words),
                    unroll=True))
lstm_model.add(Dense(nb_words))
lstm_model.add(Activation("softmax"))

lstm_model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=['acc'])

In [35]:
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    lstm_model.fit(Xtrain, Ytrain, validation_data=[Xval, Yval], batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)
    
    test_idx = np.random.randint(len(input_words))
    test_input = input_words[test_idx]
    print(' '.join(test_input))
    for i in range(NUM_PREDS_PER_EPOCH):
        test_s = np.zeros((1, SEQLEN))
        for i, w in enumerate(test_input):
            test_s[0, i] = word2index[w]
        pred = lstm_model.predict(test_s, verbose=0)[0]
        ypred = index2word[np.argmax(pred)]
        print(ypred, end=" ")
        test_input = test_input[1:] + [ypred]
    print()

Iteration #: 0
Train on 18998 samples, validate on 4750 samples
Epoch 1/10
18998/18998 [==============================] - 48s 3ms/step - loss: 7.9527 - acc: 0.0383 - val_loss: 7.9466 - val_acc: 0.0406
Epoch 2/10
18998/18998 [==============================] - 38s 2ms/step - loss: 7.5569 - acc: 0.0386 - val_loss: 7.9868 - val_acc: 0.0398
Epoch 3/10
18998/18998 [==============================] - 38s 2ms/step - loss: 7.3977 - acc: 0.0464 - val_loss: 8.0091 - val_acc: 0.0514
Epoch 4/10
18998/18998 [==============================] - 37s 2ms/step - loss: 7.2727 - acc: 0.0500 - val_loss: 7.9908 - val_acc: 0.0503
Epoch 5/10
18998/18998 [==============================] - 39s 2ms/step - loss: 7.1641 - acc: 0.0520 - val_loss: 7.9787 - val_acc: 0.0526
Epoch 6/10
18998/18998 [==============================] - 38s 2ms/step - loss: 7.0634 - acc: 0.0551 - val_loss: 8.0592 - val_acc: 0.0547
Epoch 7/10
18998/18998 [==============================] - 39s 2ms/step - loss: 6.9818 - acc: 0.0572 - val_loss: 8.

18998/18998 [==============================] - 55s 3ms/step - loss: 2.6803 - acc: 0.6671 - val_loss: 9.9594 - val_acc: 0.0303
Epoch 9/10
18998/18998 [==============================] - 56s 3ms/step - loss: 2.6894 - acc: 0.6660 - val_loss: 9.9706 - val_acc: 0.0288
Epoch 10/10
18998/18998 [==============================] - 57s 3ms/step - loss: 2.6959 - acc: 0.6680 - val_loss: 10.0172 - val_acc: 0.0278
внесла индейку борменталь налил филиппу филипповичу красного вина и предложил
шарикову я не ней я лучше меня без за со тебе но подумал они в же весь была была трубку и не ответил крикнул знаете вас вас совершенно вас борменталь совершенно конечно дверь совершенно он меня надо раза борменталь что от это время три быть в приёмной с вдруг только не шариков это и из спросил у и ним ним сбоку и фёдор шарикову дело из этот меня шарикова как это ну за шариков зина двух будет филипп филиппович что как на как дарья борменталь его до до с борменталь шарикова с тотчас преображенский и и и слышал не до 

18998/18998 [==============================] - 52s 3ms/step - loss: 2.9581 - acc: 0.6683 - val_loss: 10.3405 - val_acc: 0.0255
Epoch 8/10
18998/18998 [==============================] - 56s 3ms/step - loss: 2.9581 - acc: 0.6674 - val_loss: 10.3474 - val_acc: 0.0261
Epoch 9/10
18998/18998 [==============================] - 51s 3ms/step - loss: 2.9572 - acc: 0.6684 - val_loss: 10.3426 - val_acc: 0.0238
Epoch 10/10
18998/18998 [==============================] - 51s 3ms/step - loss: 2.9526 - acc: 0.6690 - val_loss: 10.3489 - val_acc: 0.0242
отливали в ржавый табачный цвет морщины расползались на лице у
но но шарик от был как очень пока говорите что борменталь вошёл у и рукой этого а лицо ты на пёс калоши профессор ничего молвил зина него вас а и это профессора ещё он лет о и у двух и ним начал вот филипп филиппович в вдруг уже нельзя швондера но а не никогда в видно мне я вы туда ли не ничего ничего что я я не где а о всего из этот раз а потому потому на где мне зина если что с делать ведь 

Epoch 7/10
18998/18998 [==============================] - 23s 1ms/step - loss: 2.8609 - acc: 0.6955 - val_loss: 10.6834 - val_acc: 0.0244
Epoch 8/10
18998/18998 [==============================] - 24s 1ms/step - loss: 2.8600 - acc: 0.6961 - val_loss: 10.6637 - val_acc: 0.0244
Epoch 9/10
18998/18998 [==============================] - 24s 1ms/step - loss: 2.8616 - acc: 0.6953 - val_loss: 10.6675 - val_acc: 0.0234
Epoch 10/10
18998/18998 [==============================] - 23s 1ms/step - loss: 2.8620 - acc: 0.6960 - val_loss: 10.6780 - val_acc: 0.0217
в злобе то ли в тяжёлом упадке всё было скучно
ладно будете вы моё или раз многоуважаемый они зина колбасу лет филипп филиппович не через глазами хвост с ему вниз из не этой преображенского через в шарикова спросил когда когда вы несколько в нетерпеливо ли на зина глазами вдруг я я и молвил пёс из то спросил шариков человек же мне зина это хорошо можете и борменталь вас лет севильи не не субъект как не так с пришли затем очень покраснел по ним

In [36]:
evaluation = evaluate_model(lstm_model)
print(evaluation)

417739.7225150973


In [37]:
something = text(lstm_model)

и до того и что половина их тут же была
вас лестнице дело филипп борменталь сидел гипофиз будто блюдо его он красный что шарикову кусок руки хотите вас он когда потрудитесь же комнату воду повторил и человек очень же никуда и вы надо как слова операции по кабинет на подумал слова он крикнул гипофиз он во шарик трубку не следует столовой запах к про бок над на зубы жизнь момент потолком вошёл на вынул какой и стоял за чёрт ним галстук борменталь ещё меня несмотря и к неприязненно бы доктор я собой шариков больше позвонил филипп филипп филипповича ну к он разговор под подозрительно воскликнул виноват почему как с собой две пса но отчаянно шарика за цирк бок подумал так борменталь бросился повесил как даже с филипп шариков вы была позвонил совершенно гражданин на бок к носа заглянул к полу и учёный совершенно было просто он подумал их у четверо был и сквозь просил всякие из к про эта время говорит одним совета слово дверь господин точно эту то ещё я нос этой минут виноват сидел его бормен

### GRU Embedding 50

In [23]:
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 10
NUM_PREDS_PER_EPOCH = 100

gru_model = Sequential()
gru_model.add(Embedding(nb_words, 50, input_length=SEQLEN))
gru_model.add(GRU(HIDDEN_SIZE, return_sequences=False,
                    input_shape=(SEQLEN, nb_words),
                    unroll=True))
gru_model.add(Dense(nb_words))
gru_model.add(Activation("softmax"))

gru_model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=['acc'])

In [24]:
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    gru_model.fit(Xtrain, Ytrain, validation_data=[Xval, Yval], batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)
    
    test_idx = np.random.randint(len(input_words))
    test_input = input_words[test_idx]
    print(' '.join(test_input))
    for i in range(NUM_PREDS_PER_EPOCH):
        test_s = np.zeros((1, SEQLEN))
        for i, w in enumerate(test_input):
            test_s[0, i] = word2index[w]
        pred = gru_model.predict(test_s, verbose=0)[0]
        ypred = index2word[np.argmax(pred)]
        print(ypred, end=" ")
        test_input = test_input[1:] + [ypred]
    print()

Iteration #: 0
Train on 18998 samples, validate on 4750 samples
Epoch 1/10
18998/18998 [==============================] - 39s 2ms/step - loss: 7.9747 - acc: 0.0373 - val_loss: 7.9282 - val_acc: 0.0406
Epoch 2/10
18998/18998 [==============================] - 28s 1ms/step - loss: 7.6037 - acc: 0.0392 - val_loss: 8.0590 - val_acc: 0.0406
Epoch 3/10
18998/18998 [==============================] - 31s 2ms/step - loss: 7.4824 - acc: 0.0392 - val_loss: 8.0415 - val_acc: 0.0406: 5
Epoch 4/10
18998/18998 [==============================] - 28s 1ms/step - loss: 7.3594 - acc: 0.0487 - val_loss: 8.1272 - val_acc: 0.0541
Epoch 5/10
18998/18998 [==============================] - 27s 1ms/step - loss: 7.2490 - acc: 0.0504 - val_loss: 8.0492 - val_acc: 0.0545
Epoch 6/10
18998/18998 [==============================] - 27s 1ms/step - loss: 7.1448 - acc: 0.0510 - val_loss: 8.0663 - val_acc: 0.0554
Epoch 7/10
18998/18998 [==============================] - 28s 1ms/step - loss: 7.0445 - acc: 0.0542 - val_loss:

18998/18998 [==============================] - 27s 1ms/step - loss: 4.9739 - acc: 0.3076 - val_loss: 8.7224 - val_acc: 0.0387
Epoch 2/10
18998/18998 [==============================] - 27s 1ms/step - loss: 4.9355 - acc: 0.3162 - val_loss: 8.7540 - val_acc: 0.0402
Epoch 3/10
18998/18998 [==============================] - 27s 1ms/step - loss: 4.8945 - acc: 0.3323 - val_loss: 8.8107 - val_acc: 0.0408
Epoch 4/10
18998/18998 [==============================] - 27s 1ms/step - loss: 4.8545 - acc: 0.3412 - val_loss: 8.8351 - val_acc: 0.0400
Epoch 5/10
18998/18998 [==============================] - 27s 1ms/step - loss: 4.8135 - acc: 0.3526 - val_loss: 8.8523 - val_acc: 0.0383
Epoch 6/10
18998/18998 [==============================] - 30s 2ms/step - loss: 4.7772 - acc: 0.3638 - val_loss: 8.9069 - val_acc: 0.0411
Epoch 7/10
18998/18998 [==============================] - 29s 2ms/step - loss: 4.7405 - acc: 0.3730 - val_loss: 8.9248 - val_acc: 0.0383
Epoch 8/10
18998/18998 [============================

18998/18998 [==============================] - 27s 1ms/step - loss: 3.7986 - acc: 0.5686 - val_loss: 10.0145 - val_acc: 0.0337
Epoch 2/10
18998/18998 [==============================] - 28s 1ms/step - loss: 3.7938 - acc: 0.5677 - val_loss: 10.0152 - val_acc: 0.0345
Epoch 3/10
18998/18998 [==============================] - 28s 1ms/step - loss: 3.7948 - acc: 0.5685 - val_loss: 10.0086 - val_acc: 0.0354
Epoch 4/10
18998/18998 [==============================] - 27s 1ms/step - loss: 3.7871 - acc: 0.5695 - val_loss: 10.0287 - val_acc: 0.0337
Epoch 5/10
18998/18998 [==============================] - 26s 1ms/step - loss: 3.7848 - acc: 0.5706 - val_loss: 10.0375 - val_acc: 0.0366
Epoch 6/10
18998/18998 [==============================] - 26s 1ms/step - loss: 3.7855 - acc: 0.5692 - val_loss: 10.0446 - val_acc: 0.0328
Epoch 7/10
18998/18998 [==============================] - 26s 1ms/step - loss: 3.7827 - acc: 0.5705 - val_loss: 10.0505 - val_acc: 0.0331
Epoch 8/10
18998/18998 [=====================

18998/18998 [==============================] - 26s 1ms/step - loss: 3.7073 - acc: 0.5806 - val_loss: 10.1806 - val_acc: 0.0335
Epoch 2/10
18998/18998 [==============================] - 27s 1ms/step - loss: 3.7041 - acc: 0.5812 - val_loss: 10.1762 - val_acc: 0.0337
Epoch 3/10
18998/18998 [==============================] - 27s 1ms/step - loss: 3.7046 - acc: 0.5817 - val_loss: 10.1718 - val_acc: 0.0333
Epoch 4/10
18998/18998 [==============================] - 27s 1ms/step - loss: 3.7011 - acc: 0.5827 - val_loss: 10.1768 - val_acc: 0.0352
Epoch 5/10
18998/18998 [==============================] - 26s 1ms/step - loss: 3.6993 - acc: 0.5817 - val_loss: 10.1727 - val_acc: 0.0349
Epoch 6/10
18998/18998 [==============================] - 27s 1ms/step - loss: 3.6977 - acc: 0.5833 - val_loss: 10.1825 - val_acc: 0.0318
Epoch 7/10
18998/18998 [==============================] - 26s 1ms/step - loss: 3.6991 - acc: 0.5818 - val_loss: 10.1876 - val_acc: 0.0309
Epoch 8/10
18998/18998 [=====================

18998/18998 [==============================] - 27s 1ms/step - loss: 3.6481 - acc: 0.5902 - val_loss: 10.2668 - val_acc: 0.0343
Epoch 2/10
18998/18998 [==============================] - 28s 1ms/step - loss: 3.6475 - acc: 0.5907 - val_loss: 10.2727 - val_acc: 0.0328
Epoch 3/10
18998/18998 [==============================] - 27s 1ms/step - loss: 3.6488 - acc: 0.5899 - val_loss: 10.2827 - val_acc: 0.0316
Epoch 4/10
18998/18998 [==============================] - 28s 1ms/step - loss: 3.6462 - acc: 0.5909 - val_loss: 10.2757 - val_acc: 0.0328
Epoch 5/10
18998/18998 [==============================] - 27s 1ms/step - loss: 3.6455 - acc: 0.5895 - val_loss: 10.2677 - val_acc: 0.0328
Epoch 6/10
18998/18998 [==============================] - 27s 1ms/step - loss: 3.6416 - acc: 0.5927 - val_loss: 10.2796 - val_acc: 0.0326
Epoch 7/10
18998/18998 [==============================] - 29s 2ms/step - loss: 3.6440 - acc: 0.5913 - val_loss: 10.2946 - val_acc: 0.0337
Epoch 8/10
18998/18998 [=====================

In [32]:
evaluation = evaluate_model(gru_model)
print(evaluation)

834.2671977139929


In [33]:
something = text(gru_model)

стену сказал я руки он вернулся к столу взял с
туда и до его собой мне всего борменталь пёс преображенский что же ли у чего ко говорите себя вы неожиданно было за такое мозг два филиппа филипповича сидел женщины ванной а потом было ах столовой филипп филиппович дарьи его филипп филипповича я совершенно к псу красным какая доктор филипп пёс несколько более с телефон и спросил мне ничего не мозг через через жизни двери все их мгновенно швондера на мы господи зины а так две дело больше не согласен и о бы ним ладно это с слова ли ну шариков филипп филипповича зина фёдор тут чёрт здесь мне сколько уж уж к моя разруха до себя что он где фёдор вам более подумал преображенский ведь он этот голова ли под вошёл теперь всех часов преображенский а уже от этом чёрный полиграф его женщины были хвост ничего нет на ним борменталь филиппа филипповича он несколько сидел что мало только ещё голос но можно дома а их я или филипп петровна шариков меня в уверенно швондер никогда не собой оба можно с тут две

### GRU Embedding 100

In [36]:
gru_model = Sequential()
gru_model.add(Embedding(nb_words, 100, input_length=SEQLEN))
gru_model.add(GRU(HIDDEN_SIZE, return_sequences=False,
                    input_shape=(SEQLEN, nb_words),
                    unroll=True))
gru_model.add(Dense(nb_words))
gru_model.add(Activation("softmax"))

gru_model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=['acc'])

In [37]:
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    gru_model.fit(Xtrain, Ytrain, validation_data=[Xval, Yval], batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)
    
    test_idx = np.random.randint(len(input_words))
    test_input = input_words[test_idx]
    print(' '.join(test_input))
    for i in range(NUM_PREDS_PER_EPOCH):
        test_s = np.zeros((1, SEQLEN))
        for i, w in enumerate(test_input):
            test_s[0, i] = word2index[w]
        pred = gru_model.predict(test_s, verbose=0)[0]
        ypred = index2word[np.argmax(pred)]
        print(ypred, end=" ")
        test_input = test_input[1:] + [ypred]
    print()

Iteration #: 0
Train on 18998 samples, validate on 4750 samples
Epoch 1/10
18998/18998 [==============================] - 51s 3ms/step - loss: 7.9769 - acc: 0.0378 - val_loss: 7.9481 - val_acc: 0.0411
Epoch 2/10
18998/18998 [==============================] - 33s 2ms/step - loss: 7.5865 - acc: 0.0388 - val_loss: 8.0037 - val_acc: 0.0411
Epoch 3/10
18998/18998 [==============================] - 35s 2ms/step - loss: 7.4098 - acc: 0.0491 - val_loss: 7.9156 - val_acc: 0.0543
Epoch 4/10
18998/18998 [==============================] - 35s 2ms/step - loss: 7.2830 - acc: 0.0507 - val_loss: 7.9154 - val_acc: 0.0543
Epoch 5/10
18998/18998 [==============================] - 34s 2ms/step - loss: 7.1669 - acc: 0.0504 - val_loss: 8.0129 - val_acc: 0.0560
Epoch 6/10
18998/18998 [==============================] - 36s 2ms/step - loss: 7.0509 - acc: 0.0538 - val_loss: 8.0554 - val_acc: 0.0535
Epoch 7/10
18998/18998 [==============================] - 29s 2ms/step - loss: 6.9554 - acc: 0.0564 - val_loss: 7.

вы сегодня я кто же как бы я лет очень уж ни а и вдруг бы в от севильи до сам филипп филиппович затем он поднял стал глаз и дарья петровна вам не было никогда а вдруг борменталь и тут господин борменталь и заговорил борменталь из нельзя будет сам он лицо его с вы будут можно вдруг пёс ни и молвил борменталь и доктор на ней сам здесь хоть ещё из них филиппа филипповича нет что вы почему же так нет это уж нет уж этого вы будет уж уж а то борменталь из смотровой нет как же ещё раз 
Iteration #: 8
Train on 18998 samples, validate on 4750 samples
Epoch 1/10
18998/18998 [==============================] - 51s 3ms/step - loss: 3.7371 - acc: 0.5744 - val_loss: 9.5911 - val_acc: 0.0381
Epoch 2/10
18998/18998 [==============================] - 50s 3ms/step - loss: 3.7278 - acc: 0.5751 - val_loss: 9.6104 - val_acc: 0.0413
Epoch 3/10
18998/18998 [==============================] - 51s 3ms/step - loss: 3.7247 - acc: 0.5769 - val_loss: 9.6220 - val_acc: 0.0396
Epoch 4/10
18998/18998 [=================

18998/18998 [==============================] - 52s 3ms/step - loss: 3.5275 - acc: 0.6088 - val_loss: 10.1459 - val_acc: 0.0383
звонок пролетел по квартире лакированная дверь открылась вошёл тяпнутый вручил
филиппу филипповичу лицо и ванной вошёл в через были она квартире преображенский дверь не из них года и была бы что со позвольте куда ну тотчас кого человек говорю на боже что все слова бок он не одна пёс но кроме двух до он будет до того что борменталь и сам говорит не что вы только воскликнул филипп филиппович нужно окончательно ноги себя по несколько дама руки на вам его произнёс вы потому что очень просто они не не на самом деле но ещё не понимаю в доме да что у меня ничего ещё не сидел в москве а и 
Iteration #: 16
Train on 18998 samples, validate on 4750 samples
Epoch 1/10
18998/18998 [==============================] - 52s 3ms/step - loss: 3.5248 - acc: 0.6101 - val_loss: 10.1688 - val_acc: 0.0379
Epoch 2/10
18998/18998 [==============================] - 51s 3ms/step - loss: 3.

Epoch 9/10
18998/18998 [==============================] - 46s 2ms/step - loss: 3.4806 - acc: 0.6176 - val_loss: 10.2553 - val_acc: 0.0394
Epoch 10/10
18998/18998 [==============================] - 47s 2ms/step - loss: 3.4794 - acc: 0.6162 - val_loss: 10.2722 - val_acc: 0.0385
с ним и за ним что тронул выкуси вот бы
не позвольте филиппа филипповича ну уж поднял не понимаю и шариков я сударыня вдруг него как это вы куда куда вы знает равно за такое знаете человек я один из квартиры я могу вот что не говорите а нет угодно это бы человек и всё же знает что позвольте такое филипп филиппович очень просто же это дом воскликнул филипп филиппович я же его первый не первый иван арнольдович на все вы там только до сидел того что этой ваш как все лежал была вы бы лицо ни надо будут надо до зины как пёс все операции можно калоши да он 
Iteration #: 20
Train on 18998 samples, validate on 4750 samples
Epoch 1/10
18998/18998 [==============================] - 47s 2ms/step - loss: 3.4800 - acc: 0.6165 

Epoch 9/10
18998/18998 [==============================] - 47s 2ms/step - loss: 3.4462 - acc: 0.6231 - val_loss: 10.3063 - val_acc: 0.0387
Epoch 10/10
18998/18998 [==============================] - 48s 3ms/step - loss: 3.4455 - acc: 0.6227 - val_loss: 10.3278 - val_acc: 0.0383
голос шарикова из ванной болван коротко отозвался филипп филиппович зина
и дарья петровна в говорить не клянусь если равно я вас понимаю ответил шариков вдруг уж вас доктор борменталь умоляю вас открыл равно и тяпнутый как у двери двери вас под приёмную которого потолком смотровой чего он чтобы того что ли у них вроде рядом и тот может быть не при вами а сегодня с передней а и говорит не через уж сколько вам не кричал филипп филиппович не были и дарья петровна в тот очередь с говорить а это а не угодно ли это всё равно с позвольте я один мне не да значит филипп филиппович всё равно сколько 
Iteration #: 24
Train on 18998 samples, validate on 4750 samples
Epoch 1/10
18998/18998 [==============================] - 48

In [40]:
evaluation = evaluate_model(gru_model)
print(evaluation)

1704.8843705470351


In [41]:
something = text(gru_model)
print(something)

вас может их на не несколько и в состояние на
передней то что то фёдор не вышел доктора в квартире голос я доктор что они не есть же на шкаф а сударыня штаны по шкаф раз начал самый принимать покраснел не свою тем вина а что чего при видно самого во эту чёрт полиграф сколько года несколько этой трубку с тогда человек из она вдруг мозга не председатель они одна и филиппу надо появилась шарика глаза на филиппа стало два тебе рядом по шариков очередь будут личность личность пациентов с ванной куда же в чёрном говорю не будет с есть головой потому и они нельзя же спросил вы для вас ничего нет очень просто я не белый более сидел они этого но был закричал было нужно с говорите с хмуро вдруг сколько почему только и тут так по позвольте водки что ещё эти человека лучше и угодно но сразу ещё заговорил которое пусть филипп филипп филипповича сидел у стола извините борменталь ну стало дарья филиппович пусть вошёл борменталя чёрный из кухни квартире хорошо я очень сударыня эти его сколько будет и 

### Результаты

| сетка | hidden_size | batch_size | Embedding size | Perplexity | 
|-----|-------------------|-------------|-----------------------|----------|
| **LSTM** | 128 | 128 | 50 | 80.23315832470844 | 
| **LSTM** | 128 | 128 | 100 | 25739.492306372722 | 
| **LSTM** | 216 | 128 | 50 | 417739.7225150973 | 
| **GRU** | 128 | 128 | 50 | 834.2671977139929 | 
| **GRU** | 128 | 128 | 100 | 1704.8843705470351 | 

- Получается, что чем проще модель, тем меньше перплексия и тем лучше модель лучше генерирует тексты. Возможно, чем модель сложнее, тем и сложнее ее гипотезы, а предсказания от этого страдают.
- Если говорить об адекватности тектса: кусочками он был адекватный. Нужно принимать во внимание тот факт, что из текстов были убраны все знаки препинания, что влияет на воспринимаесть текста. Также нужно помнить о том, что в книгах были диалоги и местами мы видим проблески этих диалогов.
- возможно, адекватность генерируемого текста увеличится, если увеличить кол-во эпох на одну итерацию. По крайней мере, по ходу итераций адекватность предсказаний на основе тренировочного текста при обучении увеличивалась.

Если сравнивать модели между собой, то перплексия при размере эмбеддинга 50 LSTM меньше перплексии GRU. Однако, при размере эмбеддинга 100, ситуация меняется: у GRU перплексия меньше, и она увеличилась только в два раза при увеличении размера эмбеддинга, что нельзя сказать об LSTM. Это, возможно, объясняется тем, что GRU - упрощенная версия LSTM и не пытается усложнить гипотезы по мере усложения самой модельки.